# IBM Quantum Challenge 2021 
## Lukas Burgholzer (lukas.burgholzer@jku.at)
### Johannes Kepler University Linz

Problem Statement

In [1]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

Python imports

In [2]:
from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP, ADAM, AQGD
from qiskit.algorithms import VQE
from qiskit_nature.circuit.library import HartreeFock
from qiskit_nature.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit.opflow import Z2Symmetries
from qiskit import Aer
from IPython.display import display, clear_output

Callback function for VQE

In [3]:
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

The core of the molecule is frozen to reduce the number of qubits and orbitals 3 and 4 are removed as they do not contribute much to the resulting ground state energy. 

Identifying the orbitals that can savely be removed while retaining the desired ground energy up to the required precision was a key factor here. If the formulation of the problem is not capable of providing the required accuracy even the best ansatz cannot safe the day. In order for a frozen core solution to reach the desired accuracy it has to allow a groundstate energy that is smaller than ~1.08578.

In [4]:
# specify problem and freeze transformation
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[FreezeCoreTransformer(remove_orbitals=[3,4])])

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

Parity mapping in combination with two-qubit reduction is chosen for the converter in order to further reduce the number of qubits to consider. By additionally figuring out how symmetry reduction works, additional redundancies can be exploited.

In [5]:
# Setup the mapper and qubit converter
converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction='auto')

In case the core is frozen, orbitals 3 and 4 are frozen, and parity mapping with two-qubit reduction is chosen, there is one symmetry that can be exploited to reduce the problem size. During experimenting with all the different options it was always most important to ensure that the right sector was chosen here, since only one sector allows to obtain the desired ground energy.

With all the optimizations a 4 qubit problem results eventually. I was not successful in reducing the number of qubits further without losing the required precision for actually attaining the groundstate. Although there are some combinations yielding a two-qubit system that comes pretty damn close to the actual target value (within 0.005). I figured that if an exact solver tells me I cannot reach the desired accuracy, it is not worth trying with VQE anyway.

In [6]:
# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

z2_sector = [1]
def finder(z2_symmetries: Z2Symmetries):
    return z2_sector if not z2_symmetries.is_empty() else None
qubit_op = converter.convert(main_op, num_particles=num_particles, sector_locator=finder)

As requested by the problem statement, the corresponding Hartree-Fock state is used as an initial state.

In [7]:
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

The ansatz is certainly where the most work went into. First it started out with a TwoLocal ansatz using linear entanglement and 5 layers. I soon figured out that Y-Rotations are the way to go with respect to the parameterized gates. At that point, my system still consisted of 6 qubits. With enough layers this worked out perfectly. 

Then I started trying to reduce the layers until the optimal solution could no longer be found. This led to a reduction to 3 layers. In parallel, I tried reducing the number of qubits further which would eventually allow me to reduce the number of CNOTs in the Ansatz. Once I got the problem formulation down to 4 qubits, I started writing out the TwoLocal Ansatz per-hand.

With a lot of trial and error, I figured out that many of the CNOTs were not actually needed. At the end it came down to the 3 CNOT ansatz you see below. It is just enough that all 4 qubits can be entangled. Removing any of the remaining CNOTs leads to sub-optimal results.

In [8]:
n = qubit_op.num_qubits
qc = QuantumCircuit(n)
params = ParameterVector('a', 10)
qc.ry(params[0], 0)
qc.ry(params[1], 1)

qc.cx(0, 1)

qc.ry(params[2], 0)
qc.ry(params[3], 1)
qc.ry(params[4], 2)

qc.cx(0, 2)

qc.ry(params[5], 0)
qc.ry(params[6], 2)
qc.ry(params[7], 3)

qc.cx(2, 3)

qc.ry(params[8], 2)
qc.ry(params[9], 3)

ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

Initial parameter state as requested by the problem formulation.

In [9]:
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

The actual call to the VQE algorithm is rather unspectacular. Over time I figured out that the SLSQP algorithm converged the fastest for my specific configuration, so I stuck with it. Since the problem only consists of 4 qubits and 10 parameters the optimizer converges pretty quickly.

In [10]:
counts = []
values = []
params = []
deviation = []
algorithm = VQE(ansatz,
                optimizer=SLSQP(maxiter=1000),
                quantum_instance=Aer.get_backend('statevector_simulator'),
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result.eigenvalue)

-1.088637182001766


In [11]:
# Check your answer using following code
from qc_grader import grade_ex5
freeze_core = True # change to True if you freezed core electrons
# freeze_core = False # change to True if you freezed core electrons
grade_ex5(ansatz,qubit_op,result,freeze_core)

/Users/burgholzer/ibm-quantum-challenge-2021/venv/lib/python3.9/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/burgholzer/ibm-quantum-challenge-2021/venv/lib/python3.9/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(


Grading your answer for ex5. Please wait...


/Users/burgholzer/ibm-quantum-challenge-2021/venv/lib/python3.9/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(



Congratulations 🎉! Your answer is correct.
Your cost is 3.
Feel free to submit your answer.



In [ ]:
# Submit your answer. You can re-submit at any time.
from qc_grader import submit_ex5
submit_ex5(ansatz,qubit_op,result,freeze_core)

Compute exact result

In [13]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

# specify problem and freeze transformation
exact_problem = ElectronicStructureProblem(driver, q_molecule_transformers=[FreezeCoreTransformer()])

# Setup the mapper and qubit converter
exact_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

result_exact = exact_diagonalizer(exact_problem, exact_converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)

Exact electronic energy -1.0897823963487432


Compute chemical accuracy

In [14]:
chemical_accuracy = np.abs(result.eigenvalue - exact_energy)
print("Chemical accuracy", chemical_accuracy)

Chemical accuracy 0.0011452143469772302
